In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import joblib

In [126]:
from vincentvanbot.data import create_flat_images_db, get_data_locally, flat_images_db_download
from vincentvanbot.params import IMAGES_PATH, FLAT_IMAGES_DB_PATH_ROOT, BUCKET_NAME, BUCKET_FLAT_IMAGES_DB_FOLDER
from vincentvanbot.data import get_data_locally, flat_images_db_upload, download_images_locally
from vincentvanbot.fromtitle.data_rec import create_joined_img_df, create_flat_images_db, get_data_locally, joined_images_db_upload, joined_images_db_download
from vincentvanbot.fromtitle.predict_rec import get_closest_images_indexes, get_info_from_index,get_index_of_user_input
from vincentvanbot.fromtitle.train_rec import train_model, save_model_to_cloud
from vincentvanbot.preprocessing import preprocess_image, build_pipe_for_categorical
from tqdm import tqdm
import os

In [27]:
path = os.path.join('..','..','raw_data','catalog.csv')
df = pd.read_csv(path, encoding= 'unicode_escape') # take care of non-ASCII characters such as 'ö'

df = df[df['FORM'] == 'painting'].head(100_000)
#df["index2"] = df.index
df

,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,URL,FORM,TYPE,SCHOOL,TIMEFRAME
0,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Venus and Adonis,1574-88,"Oil on canvas, 68 x 95 cm","Fogg Art Museum, Harvard University, Cambridge",https://www.wga.hu/html/a/aachen/adonis.html,painting,mythological,German,1601-1650
1,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Allegory,1598,"Oil on copper, 56 x 47 cm","Alte Pinakothek, Munich",https://www.wga.hu/html/a/aachen/allegory.html,painting,mythological,German,1601-1650
2,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Allegory of Peace, Art and Abundance",1602,"Oil on canvas, 197 x 142 cm","The Hermitage, St. Petersburg",https://www.wga.hu/html/a/aachen/allegorz.html,painting,mythological,German,1601-1650
3,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Jupiter, Antiope and Cupid",1595-98,"Oil on copper, 31 x 21 cm","Kunsthistorisches Museum, Vienna",https://www.wga.hu/html/a/aachen/antiope.html,painting,mythological,German,1601-1650
4,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Pallas Athena, Venus and Juno",1593,"Oil on canvas, 54 x 67 cm","Museum of Fine Arts, Boston",https://www.wga.hu/html/a/aachen/athena.html,painting,mythological,German,1601-1650
...,...,...,...,...,...,...,...,...,...,...,...
50559,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)",The Holy Face,c. 1660,"Oil on canvas, 104 x 85 cm","Museo de Bellas Artes, Bilbao",https://www.wga.hu/html/z/zurbaran/2/veronica....,painting,religious,Spanish,1601-1650
50560,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)",Virgin and Child with the Infant St John the B...,1662,"Oil on canvas, 169 x 127 cm","Museo de Bellas Artes, Bilbao",https://www.wga.hu/html/z/zurbaran/2/virgin.html,painting,religious,Spanish,1601-1650
50561,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)","The Virgin Mary as a Child, Asleep",c. 1655,"Oil on canvas, 103 x 90 cm",Private collection,https://www.wga.hu/html/z/zurbaran/2/virginas....,painting,religious,Spanish,1601-1650
50562,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)",Childhood of the Virgin,1658-60,"Oil on canvas, 74 x 54 cm","The Hermitage, St. Petersburg",https://www.wga.hu/html/z/zurbaran/2/virginch....,painting,religious,Spanish,1601-1650


In [114]:
df.loc[765]

AUTHOR                                   ANCHER, Michael Peter
BORN-DIED                  (b. 1849, Rutsker, d. 1927, Skagen)
TITLE                  The Lifeboat is Taken through the Dunes
DATE                                                      1883
TECHNIQUE                          Oil on canvas, 171 x 221 cm
LOCATION                  Statens Museum for Kunst, Copenhagen
URL          https://www.wga.hu/art/a/ancher/michael/lifebo...
FORM                                                  painting
TYPE                                                     genre
SCHOOL                                                  Danish
TIMEFRAME                                            1851-1900
Name: 765, dtype: object

In [5]:
df['TECHNIQUE'] = [(c.split(",", 1))[0] for c in df['TECHNIQUE']]

## Recommender System to KNN 

In [95]:
img_db = create_flat_images_db(size=1000, path=IMAGES_PATH, dim=(36,42))
img_db.index = [int(i) for i in img_db.index] # transform index from str to int

100%|██████████████████████████████| 1000/1000 [00:50<00:00, 19.64it/s]


In [129]:
pipe = build_pipe_for_categorical()
df = get_data_locally(nrows=100_000)
df

,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,URL,FORM,TYPE,SCHOOL,TIMEFRAME
0,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Venus and Adonis,1574-88,"Oil on canvas, 68 x 95 cm","Fogg Art Museum, Harvard University, Cambridge",https://www.wga.hu/art/a/aachen/adonis.jpg,painting,mythological,German,1601-1650
1,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Allegory,1598,"Oil on copper, 56 x 47 cm","Alte Pinakothek, Munich",https://www.wga.hu/art/a/aachen/allegory.jpg,painting,mythological,German,1601-1650
2,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Allegory of Peace, Art and Abundance",1602,"Oil on canvas, 197 x 142 cm","The Hermitage, St. Petersburg",https://www.wga.hu/art/a/aachen/allegorz.jpg,painting,mythological,German,1601-1650
3,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Jupiter, Antiope and Cupid",1595-98,"Oil on copper, 31 x 21 cm","Kunsthistorisches Museum, Vienna",https://www.wga.hu/art/a/aachen/antiope.jpg,painting,mythological,German,1601-1650
4,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Pallas Athena, Venus and Juno",1593,"Oil on canvas, 54 x 67 cm","Museum of Fine Arts, Boston",https://www.wga.hu/art/a/aachen/athena.jpg,painting,mythological,German,1601-1650
...,...,...,...,...,...,...,...,...,...,...,...
50559,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)",The Holy Face,c. 1660,"Oil on canvas, 104 x 85 cm","Museo de Bellas Artes, Bilbao",https://www.wga.hu/art/z/zurbaran/2/veronica.jpg,painting,religious,Spanish,1601-1650
50560,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)",Virgin and Child with the Infant St John the B...,1662,"Oil on canvas, 169 x 127 cm","Museo de Bellas Artes, Bilbao",https://www.wga.hu/art/z/zurbaran/2/virgin.jpg,painting,religious,Spanish,1601-1650
50561,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)","The Virgin Mary as a Child, Asleep",c. 1655,"Oil on canvas, 103 x 90 cm",Private collection,https://www.wga.hu/art/z/zurbaran/2/virginas.jpg,painting,religious,Spanish,1601-1650
50562,"ZURBARÁN, Francisco de","(b. 1598, Fuente de Cantos, d. 1664, Madrid)",Childhood of the Virgin,1658-60,"Oil on canvas, 74 x 54 cm","The Hermitage, St. Petersburg",https://www.wga.hu/art/z/zurbaran/2/virginch.jpg,painting,religious,Spanish,1601-1650


In [130]:
array_transformed = pipe.fit_transform(df)
array_transformed

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [131]:
column_name = pipe.get_feature_names()

In [132]:
test_df = pd.DataFrame(array_transformed, columns=column_name, index=df.index)
test_df.loc[765]

onehot__x0_genre           1.0
onehot__x0_historical      0.0
onehot__x0_interior        0.0
onehot__x0_landscape       0.0
onehot__x0_mythological    0.0
                          ... 
onehot__x2_1651-1700       0.0
onehot__x2_1701-1750       0.0
onehot__x2_1751-1800       0.0
onehot__x2_1801-1850       0.0
onehot__x2_1851-1900       1.0
Name: 765, Length: 65, dtype: float64

In [133]:
join_images_db = pd.DataFrame(img_db.join(test_df, lsuffix='_left', rsuffix='_right'))
join_images_db 

,0,1,2,3,4,5,6,7,8,9,...,onehot__x2_1401-1450,onehot__x2_1451-1500,onehot__x2_1501-1550,onehot__x2_1551-1600,onehot__x2_1601-1650,onehot__x2_1651-1700,onehot__x2_1701-1750,onehot__x2_1751-1800,onehot__x2_1801-1850,onehot__x2_1851-1900
9733,0.749020,0.741176,0.752941,0.749020,0.741176,0.752941,0.749020,0.741176,0.752941,0.749020,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14147,0.749020,0.741176,0.752941,0.749020,0.741176,0.752941,0.749020,0.741176,0.752941,0.749020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6400,0.129412,0.156863,0.086275,0.129412,0.164706,0.082353,0.160784,0.176471,0.109804,0.149020,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,0.168627,0.125490,0.117647,0.168627,0.113725,0.109804,0.152941,0.090196,0.090196,0.129412,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42301,0.423529,0.301961,0.129412,0.262745,0.156863,0.039216,0.862745,0.709804,0.415686,0.607843,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7680,0.400000,0.282353,0.243137,0.447059,0.301961,0.188235,0.525490,0.356863,0.290196,0.509804,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35017,0.207843,0.082353,0.086275,0.309804,0.250980,0.270588,0.611765,0.545098,0.576471,0.674510,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23675,0.192157,0.200000,0.105882,0.411765,0.333333,0.101961,0.200000,0.196078,0.086275,0.278431,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34309,0.192157,0.137255,0.133333,0.156863,0.105882,0.074510,0.152941,0.109804,0.086275,0.168627,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
join_images_db.iloc[:,4550:]

,onehot__x1_Catalan,onehot__x1_Danish,onehot__x1_Dutch,onehot__x1_English,onehot__x1_Finnish,onehot__x1_Flemish,onehot__x1_French,onehot__x1_German,onehot__x1_Greek,onehot__x1_Hungarian,...,onehot__x2_1401-1450,onehot__x2_1451-1500,onehot__x2_1501-1550,onehot__x2_1551-1600,onehot__x2_1601-1650,onehot__x2_1651-1700,onehot__x2_1701-1750,onehot__x2_1751-1800,onehot__x2_1801-1850,onehot__x2_1851-1900
9733,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23675,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [135]:
df3 = create_joined_img_df(size=1000)
df3

100%|██████████████████████████████| 1000/1000 [01:07<00:00, 14.76it/s]


,0,1,2,3,4,5,6,7,8,9,...,onehot__x2_1401-1450,onehot__x2_1451-1500,onehot__x2_1501-1550,onehot__x2_1551-1600,onehot__x2_1601-1650,onehot__x2_1651-1700,onehot__x2_1701-1750,onehot__x2_1751-1800,onehot__x2_1801-1850,onehot__x2_1851-1900
162,0.611765,0.490196,0.407843,0.627451,0.505882,0.423529,0.658824,0.556863,0.466667,0.721569,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
374,0.466667,0.368627,0.250980,0.474510,0.360784,0.235294,0.454902,0.356863,0.239216,0.568627,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
412,0.207843,0.200000,0.141176,0.250980,0.282353,0.231373,0.258824,0.286275,0.254902,0.239216,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566,0.109804,0.101961,0.156863,0.172549,0.160784,0.188235,0.137255,0.129412,0.141176,0.231373,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
572,0.584314,0.392157,0.333333,0.498039,0.278431,0.227451,0.549020,0.376471,0.278431,0.607843,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50329,0.392157,0.384314,0.396078,0.635294,0.650980,0.647059,0.623529,0.588235,0.592157,0.607843,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50467,0.584314,0.650980,0.627451,0.694118,0.705882,0.670588,0.662745,0.682353,0.666667,0.752941,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
50498,0.239216,0.196078,0.180392,0.384314,0.364706,0.450980,0.486275,0.450980,0.556863,0.552941,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50507,0.101961,0.094118,0.105882,0.109804,0.109804,0.117647,0.086275,0.078431,0.090196,0.074510,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
from vincentvanbot.fromtitle.data_rec import joined_images_db_download
train_df = joined_images_db_download(size=1000, source='local', rm=False)
train_model(train_df)
#save_model_to_cloud(rm=True)

In [128]:
user_input = "Allegory; AACHEN, Hans von; 1598"
user_idx = get_index_of_user_input(user_input, get_data_locally, case=False)
#img_db = create_flat_images_db(size=100_000, path=IMAGES_PATH, dim=(36,42))
user_input_transformed = create_joined_img_df(size=100).iloc[[user_idx]]
indexes = get_closest_images_indexes(user_input_transformed)
urls = get_info_from_index(indexes)
print(urls)

100%|██████████████████████████████| 100/100 [00:03<00:00, 25.51it/s]


=> recommender_model.joblib downloaded from storage
=> rec_train_indexes.joblib downloaded from storage
['https://www.wga.hu/art/a/abbate/torfani2.jpg', 'https://www.wga.hu/art/a/abbate/torfani1.jpg', 'https://www.wga.hu/art/a/adam1/battle.jpg']


In [37]:
from vincentvanbot.data import get_data_locally
df = get_data_locally(nrows=100_000)
pipe = build_pipe_for_categorical()
array_transformed = pipe.fit_transform(df)
print(pipe.get_feature_names())

['onehot__x0_genre', 'onehot__x0_historical', 'onehot__x0_interior', 'onehot__x0_landscape', 'onehot__x0_mythological', 'onehot__x0_other', 'onehot__x0_portrait', 'onehot__x0_religious', 'onehot__x0_still-life', 'onehot__x0_study', 'onehot__x1_American', 'onehot__x1_Austrian', 'onehot__x1_Belgian', 'onehot__x1_Bohemian', 'onehot__x1_Catalan', 'onehot__x1_Danish', 'onehot__x1_Dutch', 'onehot__x1_English', 'onehot__x1_Finnish', 'onehot__x1_Flemish', 'onehot__x1_French', 'onehot__x1_German', 'onehot__x1_Greek', 'onehot__x1_Hungarian', 'onehot__x1_Irish', 'onehot__x1_Italian', 'onehot__x1_Netherlandish', 'onehot__x1_Norwegian', 'onehot__x1_Other', 'onehot__x1_Polish', 'onehot__x1_Portuguese', 'onehot__x1_Russian', 'onehot__x1_Scottish', 'onehot__x1_Spanish', 'onehot__x1_Swedish', 'onehot__x1_Swiss']


In [ ]:
df.columns

In [ ]:
df = joblib.load("/Users/juliaschmitt/code/JulSchmitt/vincentvanbot/raw_data/joined_resized_images_100.joblib")
df.isnull()

In [ ]:
df.isnull().sum()

In [ ]:
for word in lst:
    if df[df['TITLE'].str.contains(lst[0])], df[df['AUTHO'].str.contains(lst[1])], df[df['DATE'].str.contains(lst[2])]
    return df[df['TITLE'].str.contains(lst[0])][:1]

matched_row = df[df['TITLE'].str.contains(lst[0])][:1]
    user_idx = matched_row.index[0]

In [ ]:
def search_title(regex: str, df, case=False):
    """User input is title of image as a string. Search title column of original df, return respective row(s) with any matches."""
    matched_row = df[df['TITLE'].str.contains(regex, regex=True, case=case, na=False)][:1]
    return matched_row

search_title("art", df, case=False)  

In [ ]:
def download_and_preprocess_user_img():
    """Get local path (download), correlating index and apply preprocessing function. Returns flat version of image vector."""
    user_df = search_title(regex: str, df, case=False)
    download_single_image(user_df)
    path = "raw_data/images/..."
    img = load_img(path,target_size=(100,100), interpolation='nearest')
    user_input_transformed = preprocess_image(img, dim=(100,100))
    return user_input_transformed


In [ ]:
# input transformed version of user input to trained KNN
# catalog csv get url - download image locally or request library 


### PCA - check visually with different components

In [ ]:
lfw_path = "/Users/juliaschmitt/code/JulSchmitt/vincentvanbot/raw_data/images"

images = []
for root, dirnames, filenames in os.walk(lfw_path):
    for filename in fnmatch.filter(filenames, '*.jpg'):
        images.append(os.path.join(root, filename))

n = len(images)
print("loaded %d images" % n)

In [ ]:
# random selection of images

img = [Image.open(images[int(n*random.random())]) for i in range(8)]
#img[1] = np.concatenate([np.array(im.getdata()).reshape((im.size[0], im.size[1], 3)) for im in img ], axis=1)
#plt.figure(figsize=(16,2))
#plt.imshow(Image.fromarray(img.astype('uint8')))


#concat_image = np.concatenate([np.asarray(t) for t in img], axis=1)

#img = [[X[idx].reshape((w,h,3))] for idx in np.random.randint(n, size=8)]
#img = np.concatenate([np.asarray(t) for t in img], axis=1)
#plt.figure(figsize=(16,8))
#plt.imshow(Image.fromarray(img_all.astype('uint8')))

#img = img.resize((int(img.width * 100 / img.height), 100))

In [ ]:
n_components = 50
pca = PCA(n_components=n_components, svd_solver='randomized', whiten=True)
pca.fit(X)

In [ ]:
Xp = pca.transform(df)
Xp.shape

In [ ]:
Xt = pca.inverse_transform(Xp)
Xt = np.clip(Xt, 0, 255)

In [ ]:
w = h = 100

In [ ]:
test_1 = X[1].reshape((w,h,3)) #for idx in np.random.randint(n, size=8)][1]
test_2 = Xt[1].reshape((w,h,3)) #for idx in np.random.randint(n, size=8)][1]
stack_img = np.hstack((test_1, test_2))
plt.figure(figsize=(32,4))
plt.imshow(stack_img)


In [ ]:
test_x = [X[idx].reshape((w,h,3)) for idx in np.random.randint(n, size=8)][1]
plt.imshow(test_x)

In [ ]:
test_xt = [Xt[idx].reshape((w,h,3)) for idx in np.random.randint(n, size=8)][1]
plt.imshow(test_xt)

In [ ]:
npca = [100, 50, 10, 2, 1]
Xt = []
for p in npca:
    print("computing PCA for %d components"%p)
    pca = PCA(n_components=p, svd_solver='randomized', whiten=True)
    pca.fit(X)
    Xp = pca.transform(X)
    Xt.append(np.clip(pca.inverse_transform(Xp), 0, 255))

In [ ]:
idxs = np.random.randint(n, size=3)
for i,idx in enumerate(idxs):
    img_orig = X[idx].reshape((w,h,3))
    img_reconstructed = np.concatenate([Xt[p][idx].reshape((w,h,3)) for p in range(len(npca))], axis=1)
    plt.figure(figsize=(2,2))
    plt.imshow(Image.fromarray(img_orig.astype('uint8')))
    plt.figure(figsize=(16,2))
    plt.imshow(Image.fromarray(img_reconstructed.astype('uint8')))


In [ ]:
img_pairs = [[X[idx].reshape((w,h,3)), Xt[idx].reshape((w,h,3))] for idx in np.random.randint(n, size=8)]
img_all = np.concatenate([np.concatenate([p[0], p[1]]) for p in img_pairs], axis=1)
plt.figure(figsize=(32,8))
plt.imshow(Image.fromarray(img_all.astype('uint8')))
